# WeRateDogs Tweeter Archive

## Gather

In [1]:
import pandas as pd
import numpy as np
import requests
#import tweepy

In [2]:
tweet = pd.read_csv('twitter-archive-enhanced.csv')

In [ ]:
###
r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
if r.status_code == 200:
    with open('image_predictions.tsv', 'wb') as f:
        f.write(r.content)

In [3]:
image = pd.read_csv('image_predictions.tsv', sep = '\t')

In [ ]:
###
consumer_key = 'X4Mu5bFyMBpjjstVS4XbYjZcL'
consumer_secret = '6nXJBPj6kBPzT0zP45fZnjiAOuMTN5OEBS3tQ7xKbfIT4NuIst'
access_token = '4366384514-l6MKA0qiPnv018rq4aSQPQd5TODG6Ko4gNeBlSA'
access_secret = '6HcJzhQoY9fZaTfj8p5XXyBdCjNsKiB8SUURkRmSQEh3H'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True )

In [ ]:
###
from timeit import default_timer as timer

start = timer()
tweet_id = []
retweet_count = []
favorite_count = []

for ids in image.tweet_id:
    try:
        tweet = api.get_status(ids, tweet_mode='extended')
        tweet_id.append(ids)
        retweet_count.append(tweet.retweet_count)
        favorite_count.append(tweet.favorite_count)
    except Exception as e:
        print(str(e))
        
end = timer()
print(end - start)    

In [ ]:
###
file = open("tweet_json.txt", "w")
for index in range(len(tweet_id)):
    file.write(str(tweet_id[index]) + " " + str(retweet_count[index]) + " " + str(favorite_count[index]) + "\n")
file.close()

In [4]:
info = pd.read_csv('tweet_json.txt', sep=" ", header=None)
info.columns = ['tweet_id','retweet_count','favorite_count']

## Assess

In [ ]:
tweet.head(10)

In [ ]:
image.head(10)

In [ ]:
info.head(10)

In [ ]:
tweet.info()

In [ ]:
image.info()

In [ ]:
info.info()

In [ ]:
all_columns = pd.Series(list(tweet) + list(image) + list(info))
all_columns[all_columns.duplicated()]

In [ ]:
list(tweet)

In [ ]:
list(image)

In [ ]:
tweet[tweet['expanded_urls'].isnull()]

In [ ]:
tweet.describe()

In [ ]:
image.describe()

In [ ]:
info.describe()

In [ ]:
tweet.sample(5)

In [ ]:
tweet.name.value_counts()

In [ ]:
tweet.source.value_counts()

In [ ]:
image.jpg_url.value_counts()

In [ ]:
tweet[tweet.name.duplicated()]

In [ ]:
tweet.rating_numerator.sort_values()

In [ ]:
tweet.rating_denominator.sort_values()

In [ ]:
len(tweet[(tweet['doggo'] == 'doggo')]) + len(tweet[(tweet['floofer'] == 'floofer')])  + len(tweet[(tweet['pupper'] == 'pupper')]) 
+ len(tweet[(tweet['puppo'] == 'puppo')]) 

#### Quality
##### `tweet` table
- in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, and retweeted_status_user_id are int not float
- Erroneous datatypes (retweeted_status_timestamp, timestamp, text, and source columns) 
- source has four categories
- name sometimes lower case, captial letter other times
- Missing expanded_urls (2297 instead of 2356)
- Missing in_reply_to_status_id and in_reply_to_user_id (78 instead of 2356)
- Missing retweeted_status_id, retweeted_status_id, retweeted_status_id (181 instead of 2356)
- actually, by, his, my, one, the, very are not dog name
- Multiple records for 'a, an' as a name
- Some rating_numerator and rating_denominator are extremely high
- Some rows have rating_numerator and rating_denominator but do not have expanded_urls

##### `image` table
- Duplicates in jpg_url
- Lowercase p1, p2, p3
- Missing records (2075 instead of 2356)

##### `info` table
- Missing records (2069 instead of 2075)

#### Tidiness
- Merge doggo, pupper, puppo, and floofer columns into one column in `tweet` table
- `info` table should be part of the `image` table

## Clean

In [5]:
tweet_clean = tweet.copy()
image_clean = image.copy()
info_clean = info.copy()

### Missing Data

#### `tweet`: Missing expanded_urls (2297 instead of 2356), Missing in_reply_to_status_id and in_reply_to_user_id (78 instead of 2356), Missing retweeted_status_id, retweeted_status_id, retweeted_status_id (181 instead of 2356)

##### Define
- Remove tweets that don't have image url
- Remove replied tweets
- Remove retweets

##### Code

In [6]:
tweet_clean = tweet_clean[~tweet_clean.expanded_urls.isnull()]

##### Test

In [7]:
tweet_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,8.924210e+17,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,8.921770e+17,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,8.918150e+17,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,8.916900e+17,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,8.913280e+17,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


##### Code

In [8]:
tweet_clean = tweet_clean[tweet_clean['in_reply_to_status_id'].isnull()]

##### Test

In [9]:
len(tweet_clean)

2274

##### Code

In [10]:
tweet_clean = tweet_clean[tweet_clean['retweeted_status_id'].isnull()]

##### Test

In [11]:
len(tweet_clean)

2094

##### Code

In [12]:
drop_columns = ['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 
                'retweeted_status_user_id', 'retweeted_status_timestamp']

tweet_clean = tweet_clean.drop(labels = drop_columns, axis = 1)

##### Test

In [13]:
tweet_clean.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,8.924210e+17,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,8.921770e+17,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,8.918150e+17,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,8.916900e+17,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,8.913280e+17,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


### Quality

#### `tweet`: Erroneous datatypes (tweet_id, timestamp, text, and source columns) , Source has four categories

##### Define
- Change tweet_id to int data type
- Change timestamp to date data type
- Change text to string data type
- Change source to category data type
- Replace the four category source with short version

##### Code

In [14]:
tweet_clean.tweet_id = tweet_clean.tweet_id.astype(int) 
tweet_clean.text = tweet_clean.text.astype(str)
tweet_clean.source = tweet_clean.source.astype('category')
tweet_clean.timestamp = pd.to_datetime(tweet_clean.timestamp)

In [15]:
tweet_clean.source = tweet_clean.source.replace('<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'Twitter for iPhone')
tweet_clean.source = tweet_clean.source.replace('<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>', 'Vine Make a Scene')
tweet_clean.source = tweet_clean.source.replace('<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'Twitter Web Client')
tweet_clean.source = tweet_clean.source.replace('<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>', 'Tweet Deck')

##### Test

In [16]:
tweet_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2094 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2094 non-null int64
timestamp             2094 non-null datetime64[ns]
source                2094 non-null object
text                  2094 non-null object
expanded_urls         2094 non-null object
rating_numerator      2094 non-null int64
rating_denominator    2094 non-null int64
name                  2094 non-null object
doggo                 2094 non-null object
floofer               2094 non-null object
pupper                2094 non-null object
puppo                 2094 non-null object
dtypes: datetime64[ns](1), int64(3), object(8)
memory usage: 212.7+ KB


In [17]:
tweet_clean.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892421000000000000,2017-08-01 16:23:56,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177000000000000,2017-08-01 00:17:27,Twitter for iPhone,This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815000000000000,2017-07-31 00:18:03,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891690000000000000,2017-07-30 15:58:51,Twitter for iPhone,This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891328000000000000,2017-07-29 16:00:24,Twitter for iPhone,This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


#### `tweet`: Some rating_numerator and rating_denominator are extremely high

##### Define
- Calculate the rate based on rating_numerator divide rating_denominator
- Remove the rates greater than 2.0

##### Code

In [18]:
tweet_clean['rate'] = tweet_clean.rating_numerator / tweet_clean.rating_denominator

In [19]:
for dog_rate in tweet_clean.rate:
    if dog_rate > 2.0:
        print(dog_rate)

3.42857142857
7.5
2.7
177.6
2.6
42.0


In [20]:
tweet_clean = tweet_clean[tweet_clean.rate < 2]

##### Test

In [21]:
tweet_clean.rate.sort_values()

315     0.0
1869    0.1
2261    0.1
2338    0.1
2091    0.1
2310    0.2
2237    0.2
2326    0.2
2246    0.2
2079    0.2
1764    0.2
1165    0.2
1920    0.2
1761    0.2
2349    0.2
2305    0.3
2239    0.3
765     0.3
2136    0.3
1898    0.3
1189    0.3
1947    0.3
1478    0.3
1249    0.3
2202    0.3
1938    0.3
2183    0.3
1314    0.3
1692    0.3
1836    0.3
       ... 
141     1.4
1609    1.4
117     1.4
347     1.4
35      1.4
490     1.4
76      1.4
866     1.4
187     1.4
83      1.4
418     1.4
199     1.4
788     1.4
924     1.4
404     1.4
448     1.4
110     1.4
56      1.4
209     1.4
449     1.4
214     1.4
758     1.4
1053    1.4
41      1.4
372     1.4
369     1.4
436     1.4
164     1.4
395     1.4
533     1.4
Name: rate, Length: 2088, dtype: float64

#### `tweet`: Multiple records for 'a, an' as a name, 'actually, by, his, my, one, the, very' are not dog name

##### Define
- Filter the text column to check if 'a','an', etc have corresponding name
- If not, change the name to 'None'

##### Code

In [ ]:
Exception_names = ['a', 'an','actually', 'by', 'his', 'my', 'one', 'the', 'very']

In [ ]:
for names in tweet_clean.name:
    if names in Exception_names:
#            print(names)
        print(tweet_clean.loc[tweet_clean['name'] == names, 'text'])
# for descrption in tweet_clean.text:
#     print(descrption)

##### Test

#### `image`: Duplicates in jpg_url

##### Define
- Find the duplicates in jpg_url
- Remove the duplicate rows

##### Code

In [37]:
image_clean = image_clean[~image_clean.duplicated(['jpg_url'], keep=False)]

##### Test

In [38]:
image_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1943 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    1943 non-null int64
jpg_url     1943 non-null object
img_num     1943 non-null int64
p1          1943 non-null object
p1_conf     1943 non-null float64
p1_dog      1943 non-null bool
p2          1943 non-null object
p2_conf     1943 non-null float64
p2_dog      1943 non-null bool
p3          1943 non-null object
p3_conf     1943 non-null float64
p3_dog      1943 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 157.5+ KB


#### `tweet`: Name sometimes lower case, captial letter other times
#### `image`: Lowercase p1, p2, p3

##### Define
- Lower case name column in 'tweet' table
- Lower case p1, p2, p3 columns in 'image' table

##### Code

In [39]:
tweet_clean['name'] = tweet_clean['name'].str.lower()
image_clean['p1'] = image_clean['p1'].str.lower()
image_clean['p2'] = image_clean['p2'].str.lower()
image_clean['p3'] = image_clean['p3'].str.lower()

##### Test

In [45]:
tweet_clean.sample(5)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,rate
1612,685322000000000000,2016-01-08 04:46:13,Twitter for iPhone,This is Rocky. He sleeps like a psychopath. 10...,https://twitter.com/dog_rates/status/685321586...,10,10,rocky,None,None,None,None,1.0
891,759557000000000000,2016-07-31 01:12:26,Twitter for iPhone,This is Emma. She can't believe her last guess...,https://twitter.com/dog_rates/status/759557299...,10,10,emma,None,None,None,None,1.0
2347,666057000000000000,2015-11-16 00:55:59,Twitter for iPhone,My oh my. This is a rare blond Canadian terrie...,https://twitter.com/dog_rates/status/666057090...,9,10,a,None,None,None,None,0.9
1070,740359000000000000,2016-06-08 01:45:19,Twitter for iPhone,This is Chase. He's in a predicament. 9/10 hel...,https://twitter.com/dog_rates/status/740359016...,9,10,chase,None,None,None,None,0.9
353,831309000000000000,2017-02-14 01:09:44,Twitter for iPhone,This is Scooter and his son Montoya. Scooter ...,https://twitter.com/dog_rates/status/831309418...,12,10,scooter,None,None,None,None,1.2


In [44]:
image_clean.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1842,838476387338051585,https://pbs.twimg.com/media/C6Ld0wYWgAQQqMC.jpg,3,great_pyrenees,0.997692,True,kuvasz,0.001001,True,newfoundland,0.000405,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
1167,735648611367784448,https://pbs.twimg.com/media/CjWMezdW0AErwU3.jpg,1,pembroke,0.462594,True,seat_belt,0.261854,False,cardigan,0.151698,True
303,671511350426865664,https://pbs.twimg.com/media/CVGwAh-W4AAIHJz.jpg,1,hermit_crab,0.625409,False,tick,0.127333,False,snail,0.097916,False
874,698178924120031232,https://pbs.twimg.com/media/CbBuBhbWwAEGH29.jpg,1,chesapeake_bay_retriever,0.351868,True,malinois,0.207753,True,labrador_retriever,0.154606,True


### Tidiness

#### Merge doggo, pupper, puppo, and floofer columns into one column in `tweet` table

##### Define
*Your definition here. Hint: use pandas' [melt function](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html) and [`str.split()` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.split.html). Here is an excellent [`melt` tutorial](https://deparkes.co.uk/2016/10/28/reshape-pandas-data-with-melt/).*

Melt the auralin and novodra columns to a treatment and a dose column (dose will still contain both start and end dose at this point). Then split the dose column on ' - ' to obtain start_dose and end_dose columns. Drop the intermediate dose column.

##### Code

In [ ]:
# Your cleaning code here
treatments_clean = pd.melt(treatments_clean, id_vars=['given_name', 'surname', 'hba1c_start', 'hba1c_end', 'hba1c_change'],
                           var_name='treatment', value_name='dose')
treatments_clean = treatments_clean[treatments_clean.dose != "-"]
treatments_clean['dose_start'], treatments_clean['dose_end'] = treatments_clean['dose'].str.split(' - ', 1).str
treatments_clean = treatments_clean.drop('dose', axis=1)

##### Test

In [ ]:
# Your testing code here
treatments_clean.head()

#### `info` table should be part of the `image` table

##### Define
Merge the info table to the treatments table, joining on tweet_id.

##### Code

In [46]:
image_clean = pd.merge(image_clean, info_clean,
                            on=['tweet_id'], how='left')

##### Test

In [49]:
image_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1943 entries, 0 to 1942
Data columns (total 14 columns):
tweet_id          1943 non-null int64
jpg_url           1943 non-null object
img_num           1943 non-null int64
p1                1943 non-null object
p1_conf           1943 non-null float64
p1_dog            1943 non-null bool
p2                1943 non-null object
p2_conf           1943 non-null float64
p2_dog            1943 non-null bool
p3                1943 non-null object
p3_conf           1943 non-null float64
p3_dog            1943 non-null bool
retweet_count     1942 non-null float64
favorite_count    1942 non-null float64
dtypes: bool(3), float64(5), int64(2), object(4)
memory usage: 187.8+ KB


In [50]:
image_clean[image_clean['retweet_count'].isnull()]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count
1712,837012587749474308,https://pbs.twimg.com/media/C52pYJXWgAA2BEf.jpg,1,toilet_tissue,0.186387,False,cowboy_hat,0.158555,False,sombrero,0.14947,False,NaN,NaN


In [51]:
image_clean.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count
188,669375718304980992,https://pbs.twimg.com/media/CUoZqaqWcAAA2MQ.jpg,1,airedale,0.168762,True,norfolk_terrier,0.107479,True,lakeland_terrier,0.097846,True,771.0,1394.0
290,671347597085433856,https://pbs.twimg.com/media/CVEbFDRWsAAkN_7.jpg,1,picket_fence,0.382918,False,rain_barrel,0.108809,False,plastic_bag,0.038878,False,462.0,1012.0
680,684200372118904832,https://pbs.twimg.com/media/CX7EkuHWkAESLZk.jpg,1,llama,0.681347,False,ram,0.120142,False,hog,0.043686,False,1143.0,2347.0
1640,825147591692263424,https://pbs.twimg.com/media/C3ODWpfXAAAP1fb.jpg,1,pekinese,0.354823,True,pomeranian,0.245390,True,toy_poodle,0.136545,True,5140.0,19890.0
1706,836001077879255040,https://pbs.twimg.com/media/C5oSiskU0AE8sJ_.jpg,4,samoyed,0.963558,True,white_wolf,0.019848,False,malamute,0.005904,True,4803.0,20570.0


### Quality

<font color='red'>Complete the remaining "Quality" **Define, Code, and Test** sequences after watching the *"Cleaning for Quality"* video.</font>

#### Multiple phone number formats

##### Define
*Your definition here. Hint: helpful [Stack Overflow answer](https://stackoverflow.com/a/123681).*

Strip all " ", "-", "(", ")", and "+" and store each number without any formatting. Pad the phone number with a 1 if the length of the number is 10 digits (we want country code).

##### Code

In [ ]:
# Your cleaning code here
patients_clean.phone_number = patients_clean.phone_number.str.replace(r'\D+', '').str.pad(11, fillchar='1')

##### Test

In [ ]:
# Your testing code here
patients_clean.phone_number.head()

#### Default John Doe data

##### Define
*Your definition here. Recall that it is assumed that the data that this John Doe data displaced is not recoverable.*

Remove the non-recoverable John Doe records from the patients table.

##### Code

In [ ]:
# Your cleaning code here
patients_clean = patients_clean[patients_clean.surname != 'Doe']

##### Test

In [ ]:
# Your testing code here
# Should be no Doe records
patients_clean.surname.value_counts()

In [ ]:
# Should be no 123 Main Street records
patients_clean.address.value_counts()

#### Multiple records for Jakobsen, Gersten, Taylor

##### Define
*Your definition here.*

Remove the Jake Jakobsen, Pat Gersten, and Sandy Taylor rows from the patients table. These are the nicknames, which happen to also not be in the treatments table (removing the wrong name would create a consistency issue between the patients and treatments table). These are all the second occurrence of the duplicate. These are also the only occurences of non-null duplicate addresses.

##### Code

In [ ]:
# Your cleaning code here
# tilde means not: http://pandas.pydata.org/pandas-docs/stable/indexing.html#boolean-indexing
patients_clean = patients_clean[~((patients_clean.address.duplicated()) & patients_clean.address.notnull())]

##### Test

In [ ]:
# Your testing code here
patients_clean[patients_clean.surname == 'Jakobsen']

In [ ]:
patients_clean[patients_clean.surname == 'Gersten']

In [ ]:
patients_clean[patients_clean.surname == 'Taylor']